In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(X, y)

### 1. simplest flow

In [ ]:
# Load layers
from tensorflow.keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Create a dense layer and connect the dense layer to the input_tensor in one step
output_tensor = Dense(1)(input_tensor)

# Build the model
from tensorflow.keras.models import Model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer="adam", loss="mean_absolute_error")

In [ ]:
# Import the plotting function
from tensorflow.keras.utils import plot_model

# Summarize the model
model.summary()

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

In [ ]:
# Now fit the model
model.fit(X_train, y_train,
          epochs=1,
          batch_size=128,
          validation_split=0.1,
          verbose=True)

# Evaluate the model on the test data
print(model.evaluate(X_test, y_test, verbose=False))

### 2. Models with 2 inputs

#### 2.1 Categorical embedding


- when representing a high canality categorical features
- embedding will learn values for (each value of categorial features)

<img src="files/embedding_layer.png" style="height:50px" />

In [ ]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season["team_1"]).shape[0]

# Create an embedding layer, output is interger --> dim is 1, input_length is integer --> length = 1 
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1, 
                        input_length=1, 
                        name='Team-Strength')

In [1]:
# Imports
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

#### 2.2 shared layers

- shared layer will use the same weight for different inputs

<img src="files/shared_layer_1.png" style="height:200px"/>

In [ ]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input(shape=(1,), name="Team-1-In")

# Separate input layer for team 2
team_in_2 = Input(shape=(1, ), name="Team-2-In")

# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

#### 2.3 merge layers

- to join multiple inputs to single output

In [ ]:
# Imports
from keras.layers import Subtract
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model_merged = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model_merged.compile("adam", loss="mean_absolute_error")

In [ ]:
# Get the team_1 column from the regular season data
input_1 = games_season["team_1"]

# Get the team_2 column from the regular season data
input_2 = games_season["team_2"]

# Fit the model to input 1 and 2, using score diff as a target
model_merged.fit([input_1, input_2],
          games_season["score_diff"],
          epochs=1,
          batch_size=2048,
          validation_split=0.1,
          verbose=True)
# Get team_1 from the tournament data
input_1 = games_tourney["team_1"]

# Get team_2 from the tournament data
input_2 = games_tourney["team_2"]

# Evaluate the model using these inputs
print(model_merged.evaluate([input_1, input_2], games_tourney["score_diff"], verbose=False))

### 3. Models with 3 inputs

- Use Concatenate layer to combine inputs

<img src="files/3_input_layer.png"　style="height:150px"/>